In [30]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.models import resnet18
from torchvision.transforms import ToTensor, Normalize, Compose

# 定义自定义数据集
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample, label





In [103]:
import torch
from torch.utils.data import random_split, TensorDataset, DataLoader

# 创建一个形状为 (300, 3, 4, 5) 的随机特征张量和一个形状为 (300,) 的随机标签张量
features = torch.randn(300, 3, 4, 5)
labels = torch.randint(0, 3, (300,))  # 假设有 10 个类

# 将特征和标签包装在 TensorDataset 中
dataset = TensorDataset(features, labels)

# 划分数据集
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 检查 train_dataset 的样本和标签的维度
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)

# 获取第一个样本和标签
first_sample, first_label = next(iter(train_loader))
print(f"train_dataset 的样本维度是: {first_sample.shape[1:]}")
print(f"train_dataset 的标签维度是: {first_label.shape}")

# 输出 train_dataset 的整体维度信息
print(f"train_dataset 中的样本数: {len(train_dataset)}")


train_dataset 的样本维度是: torch.Size([3, 4, 5])
train_dataset 的标签维度是: torch.Size([1])
train_dataset 中的样本数: 240


In [104]:
# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [105]:
for i, value in enumerate(train_loader):
    print(i, value)

0 [tensor([[[[-0.2995, -0.7932,  0.9976,  1.1209, -0.0516],
          [ 0.6295,  0.0227,  1.0940,  0.4714,  0.3257],
          [-1.0794,  1.0486, -0.6267,  0.8861, -1.0897],
          [-1.5023,  1.0073, -1.1284,  1.0921, -0.8461]],

         [[-0.8210, -0.5820,  1.4477,  2.5089, -0.2984],
          [ 0.6298, -1.4890,  0.2860,  0.2626, -0.0836],
          [ 0.3010,  0.8257, -0.7218,  0.7714, -1.1996],
          [ 1.2969, -2.3093,  0.9216, -0.3361, -1.2674]],

         [[ 1.7777,  0.8044, -0.2151,  0.2985, -0.4174],
          [-0.5215,  1.7698, -0.0890, -0.2707, -0.2878],
          [-1.1555, -1.0338, -0.9372, -0.3259,  0.6596],
          [ 0.8937, -0.0258,  2.6149,  0.3250, -1.0999]]],


        [[[-0.8260, -0.3000, -1.9529,  0.6136,  1.2965],
          [ 0.7094,  0.9864, -0.6254,  0.0602, -1.1275],
          [ 0.0603, -1.6350,  1.2292, -2.1123,  1.0408],
          [ 0.4430,  0.8102,  1.7011,  0.7703, -0.4794]],

         [[-0.5897, -0.8042, -1.5868, -0.0158, -0.1498],
          [ 0.2978

In [106]:
# 定义ResNet模型
class SimpleResNet(nn.Module):
    def __init__(self, num_classes=3):
        super(SimpleResNet, self).__init__()
        self.resnet = resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# 初始化模型
model = SimpleResNet(num_classes=3)

# 选择损失函数
criterion = nn.CrossEntropyLoss()

# 使用Adam优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [109]:
# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        # print('yes')
        # print(inputs.shape)
        # print(labels)
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # 检查输出和标签的形状
        #print(f"Outputs shape: {outputs.shape}, Labels shape: {labels.shape}")

        # 计算损失
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")


Epoch 1/10, Loss: 0.7985
Epoch 2/10, Loss: 0.7797
Epoch 3/10, Loss: 0.7034
Epoch 4/10, Loss: 0.7606
Epoch 5/10, Loss: 0.8438
Epoch 6/10, Loss: 0.8069
Epoch 7/10, Loss: 0.7062
Epoch 8/10, Loss: 0.6146
Epoch 9/10, Loss: 0.6371
Epoch 10/10, Loss: 0.6137


In [110]:
# 测试模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.3167
